# REST API
<img src='pics/0.png' />


# Authentication & Permissions
<img src='pics/Untitled-4.jpg' />

Authentication & Permissions
Currently our API doesn't have any restrictions on who can edit or delete code authors.
We'd like to have some more advanced behavior in order to make sure that:

1. Authors are always associated with a creator.
2. Only authenticated users may create authors.
3. Only the creator of a author may update or delete it.
4. Unauthenticated requests should have full read-only access.

# models.py

In [ ]:
from django.db import models


class Author(models.Model):
    """
    Model representing an author.
    """
    owner = models.ForeignKey('auth.User', related_name='authors', on_delete=models.CASCADE)
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    date_of_birth = models.DateField(null=True, blank=True)
    date_of_death = models.DateField('Died', null=True, blank=True)

    def __str__(self):
        """
        String for representing the Model object.
        """
        return '%s, %s' % (self.last_name, self.first_name)


# serializers.py

In [ ]:
class UserSerializer(serializers.ModelSerializer):
    authors = serializers.PrimaryKeyRelatedField(many=True, queryset=models.Author.objects.all())

    class Meta:
        model = User
        fields = ('id', 'username', 'authors')

# urls.py

In [ ]:
url(r'^users/$', views.UserList.as_view()),
url(r'^users/(?P<pk>[0-9]+)/$', views.UserDetail.as_view()),

# views.py

In [ ]:
class AuthorList(generics.ListCreateAPIView):
    queryset = models.Author.objects.all()
    serializer_class = serializers.AuthorSerializer

    def perform_create(self, serializer):
        serializer.save(owner=self.request.user)
        

class AuthorDetail(generics.RetrieveUpdateDestroyAPIView):
    queryset = models.Author.objects.all()
    serializer_class = serializers.AuthorSerializer


class UserList(generics.ListAPIView):
    queryset = User.objects.all()
    serializer_class = serializers.UserSerializer


class UserDetail(generics.RetrieveAPIView):
    queryset = User.objects.all()
    serializer_class = serializers.UserSerializer

### python manage.py makemigrations

You are trying to add a non-nullable field 'owner' to author without a default; we can't do that (the database needs something to populate existing rows).

Please select a fix:

1. Provide a one-off default now (will be set on all existing rows with a null value for this column)

2. Quit, and let me add a default in models.py


In [ ]:
rm -f db.sqlite3
rm -r snippets/migrations
python manage.py makemigrations snippets
python manage.py migrate
python manage.py createsuperuser
python manage.py runserver

<img src='pics/Screenshot from 2018-03-10 11-14-16.png'>

## Associating authors with Users
Right now, if we created a author, there'd be no way of associating the user that created the author, with the author instance. The user isn't sent as part of the serialized representation, but is instead a property of the incoming request.

The way we deal with that is by overriding a .perform_create() method on our author views, that allows us to modify how the instance save is managed, and handle any information that is implicit in the incoming request or requested URL.

The create() method of our serializer will now be passed an additional 'owner' field, along with the validated data from the request.

On the AuthorList view class, add the following method:

In [ ]:
def perform_create(self, serializer):
    serializer.save(owner=self.request.user)

# add owner To AuthorSerializer

In [ ]:
class AuthorSerializer(serializers.ModelSerializer):
    # owner = serializers.ReadOnlyField(source='owner.username')

    class Meta:
        model = models.Author
        fields = ['id', 'first_name', 'last_name',
                  'date_of_birth', 'date_of_death',
                  'owner']


<img src='pics/Screenshot from 2018-03-10 13-07-45.png' />

# Make owner ReadOnly

This field is doing something quite interesting. The source argument controls which attribute is used to populate a field, and can point at any attribute on the serialized instance. It can also take the dotted notation shown above, in which case it will traverse the given attributes, in a similar way as it is used with Django's template language.

The field we've added is the untyped ReadOnlyField class, in contrast to the other typed fields, such as CharField, BooleanField etc... The untyped ReadOnlyField is always read-only, and will be used for serialized representations, but will not be used for updating model instances when they are deserialized. We could have also used CharField(read_only=True) here.

In [ ]:
owner = serializers.ReadOnlyField(source='owner.username')

<img src='pics/Screenshot from 2018-03-10 13-09-34.png' />

## Adding required permissions to views

we want to make sure that only authenticated users are able to create, update and delete code snippets.

REST framework includes a number of permission classes that we can use to restrict who can access a given view. In this case the one we're looking for is IsAuthenticatedOrReadOnly, which will ensure that authenticated requests get read-write access, and unauthenticated requests get read-only access.



In [ ]:
from rest_framework import permissions

class AuthorList(generics.ListCreateAPIView):
    queryset = models.Author.objects.all()
    serializer_class = serializers.AuthorSerializer
    permission_classes = (permissions.IsAuthenticatedOrReadOnly,)

    def perform_create(self, serializer):
        serializer.save(owner=self.request.user)


class AuthorDetail(generics.RetrieveUpdateDestroyAPIView):
    queryset = models.Author.objects.all()
    serializer_class = serializers.AuthorSerializer
    permission_classes = (permissions.IsAuthenticatedOrReadOnly,)

<img src='pics/Screenshot from 2018-03-10 13-15-05.png' />

## Project level URL

In [ ]:
urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^myapp/', include('my_app.urls', namespace='my_app')),
    url(r'^api-auth/', include('rest_framework.urls', namespace='rest_framework')),
]


<img src='pics/Screenshot from 2018-03-10 14-04-00.png' />

## Object level permissions

Really we'd like all authors to be visible to anyone,
but also make sure that only the user that created an author is able to update or delete it.

To do that we're going to need to create a custom permission.

In the the app, create a new file, custom_permissions.py

In [ ]:
from . import custom_permissions


class AuthorDetail(generics.RetrieveUpdateDestroyAPIView):
    queryset = models.Author.objects.all()
    serializer_class = serializers.AuthorSerializer
    permission_classes = (
                    permissions.IsAuthenticatedOrReadOnly,
                    custom_permissions.IsOwnerOrReadOnly
    )


## Create User

<img src='pics/Screenshot from 2018-03-10 14-27-10.png' />

## Add Author

<img src='pics/Screenshot from 2018-03-10 14-29-02.png' />

## check new author

<img src='pics/Screenshot from 2018-03-10 14-30-03.png' />

## loging with new user

### loging buttom will be add on top right of page
<img src='pics/Screenshot from 2018-03-11 15-16-23.png' />

<img src='pics/Screenshot from 2018-03-10 14-31-27.png' />

<img src='pics/Screenshot from 2018-03-11 15-21-33.png' />

### Try to delete

<img src='pics/Screenshot from 2018-03-10 14-33-01.png' />

## logging out

<img src='pics/Screenshot from 2018-03-10 14-36-34.png' />